# Double Heston Model

In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
from scipy.stats import norm

## Vanilla Option Pricing

In [ ]:
def double_heston_charfunc(u, params):
    S0 = params["S0"]
    r  = params["r"]
    q  = params["q"]
    T  = params["T"]

    # Parameters of variance process 1
    v01    = params["v01"]
    kappa1 = params["kappa1"]
    theta1 = params["theta1"]
    sigma1 = params["sigma1"]
    rho1   = params["rho1"]

    # Parameters of variance process 2
    v02    = params["v02"]
    kappa2 = params["kappa2"]
    theta2 = params["theta2"]
    sigma2 = params["sigma2"]
    rho2   = params["rho2"]

    iu = 1j * u

    def _C_D(kappa, theta, sigma, rho):
        b = kappa - rho * sigma * iu
        d = np.sqrt(b**2 + sigma**2 * (u**2 + iu))
        g = (b - d) / (b + d)
        e_dT = np.exp(-d * T)

        C = (kappa * theta / sigma**2) * (
            (b - d) * T - 2.0 * np.log((1.0 - g * e_dT) / (1.0 - g))
        )
        D = (b - d) / sigma**2 * ((1.0 - e_dT) / (1.0 - g * e_dT))
        return C, D

    C1, D1 = _C_D(kappa1, theta1, sigma1, rho1)
    C2, D2 = _C_D(kappa2, theta2, sigma2, rho2)

    C_total = iu * (r - q) * T + C1 + C2

    # φ(u) complet
    return np.exp(C_total + D1 * v01 + D2 * v02 + iu * np.log(S0))

def price_call_put(params, N=10000, U_max=1000):
    S0 = params["S0"]
    r  = params["r"]
    q  = params["q"]
    T  = params["T"]
    K  = params["K"]

    # Simpson -> N pair
    if N % 2 == 1:
        N += 1

    u  = np.linspace(1e-10, U_max, N + 1)
    du = u[1] - u[0]
    lnK = np.log(K)

    φ_u      = double_heston_charfunc(u, params)
    φ_u_im1  = double_heston_charfunc(u - 1j, params)
    φ_im1    = double_heston_charfunc(-1j, params)

    I1 = np.real(np.exp(-1j * u * lnK) * φ_u_im1 / (1j * u * φ_im1))
    I2 = np.real(np.exp(-1j * u * lnK) * φ_u / (1j * u))

    w = np.ones(N + 1)
    w[1:-1:2] = 4.0
    w[2:-2:2] = 2.0

    P1 = 0.5 + (du / (3 * np.pi)) * np.sum(w * I1)
    P2 = 0.5 + (du / (3 * np.pi)) * np.sum(w * I2)

    call = S0 * np.exp(-q * T) * P1 - K * np.exp(-r * T) * P2
    put  = K * np.exp(-r * T) * (1.0 - P2) - S0 * np.exp(-q * T) * (1.0 - P1)
    return call, put


In [11]:
params = {
    "S0": 100,
    "K" : 100,
    "r":  0.014,
    "q": 0.015,
    "v01": 0.0112,
    "kappa1": 0.2183,
    "theta1": 0.0601,
    "sigma1": 0.3171,
    "rho1": -0.5250,
    "v02": 0.0112,
    "kappa2": 0.2183,
    "theta2": 0.0601,
    "sigma2": 0.3171,
    "rho2": -0.5250,
    "T": 0.10
}
call, put = price_call_put(params)
print(f"Call Price: {call:.4f}")
print(f"Put Price: {put:.4f}")

Call Price: 1.8887
Put Price: 1.8986


## Data Extraction

In [12]:
excel_file = "market_data.xlsx"

wb = pd.read_excel(excel_file,sheet_name='NDX-2014' ,header=None,)


header_row_idx = 6
start_col = 2
end_col = 13               
n_maturities = 14

raw_header = wb.iloc[header_row_idx, start_col:end_col].tolist()
raw_data   = wb.iloc[
    header_row_idx+1 : header_row_idx+1+n_maturities,
    start_col : end_col
]


df = pd.DataFrame(
    raw_data.values[:, 1:],          
    index=raw_data.values[:, 0],    
    columns=raw_header[1:],         
)


df.columns = [f"{x:.1%}" for x in df.columns.astype(float)]


print(df.head())

S0 = wb.iat[3, 4]
print(S0)

r = wb.iat[1, 10]
print(r)
q = wb.iat[2, 10]
print(q)

       80.0%     90.0%     95.0%      97.5%   100.0%     102.5%    105.0%  \
T   3036.592  3416.166  3605.953  3700.8465  3795.74  3890.6335  3985.527   
1M     25.29     19.15     15.35       13.7    12.25      11.04     10.71   
2M     24.28     18.46     15.81      14.58    13.43      12.39     11.67   
3M     23.03     18.08     15.96      14.96     14.0       13.1     12.38   
6M     21.93     18.24     16.61      15.85    15.13      14.47     13.86   

      110.0%    120.0%  
T   4175.314  4554.888  
1M     11.46     13.56  
2M     11.52     12.55  
3M     11.83     12.06  
6M     12.87      12.4  
3795.74
0.014
0.015


## Model Calibration

In [13]:
# 1) Black–Scholes call price 

def bs_call_price(S, K, r, q, sigma, T):
    d1 = (np.log(S/K) + (r - q + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*np.exp(-q*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


# Convert maturity labels to years
def maturity_to_T(label):
    if label == 'T':
        return None
    if label.endswith('M'):
        return int(label[:-1]) / 12.0
    if label.endswith('Y'):
        return float(label[:-1])
    raise ValueError(f"Unknown maturity: {label}")

# Build market data list: (T, K, call_price)
market_data = []
strikes_abs = df.loc['T']
for mat in df.index:
    T = maturity_to_T(mat)
    if T is None:
        continue
    for pct in df.columns:
        K       = strikes_abs[pct]
        vol_imp = df.loc[mat, pct] / 100.0
        price   = bs_call_price(S0, K, r, q, vol_imp, T)
        market_data.append((T, K, price))

market_data = np.array(market_data,
                       dtype=[('T', float), ('K', float), ('price', float)])

# 4) Parameter for double Heston

def pack_params(x):
    return {
        "S0":    S0,
        "r":     r,
        "q":     q,
        # Heston factor 1
        "v01":   x[0],
        "kappa1":x[1],
        "theta1":x[2],
        "sigma1":x[3],
        "rho1":  x[4],
        # Heston factor 2
        "v02":   x[5],
        "kappa2":x[6],
        "theta2":x[7],
        "sigma2":x[8],
        "rho2":  x[9],
    }

# Residuals between model and market

def residuals(x):
    params_base = pack_params(x)
    res = []
    for T, K, mkt_price in market_data:
        params = params_base.copy()
        params.update({"T": T, "K": K})
        model_call, _ = price_call_put(params, N=2000, U_max=2000)
        res.append(model_call - mkt_price)
    return np.array(res)

#  5) Calibration via non-linear least squares 
# Initial guess for [v01, kappa1, theta1, sigma1, rho1, v02, kappa2, theta2, sigma2, rho2]
x0 = np.array([
    0.04,    # v01
    1.0,     # kappa1
    0.04,    # theta1
    0.5,     # sigma1
    -0.5,    # rho1
    0.04,    # v02
    1.0,     # kappa2
    0.04,    # theta2
    0.5,     # sigma2
    -0.5,    # rho2
])
# Bounds: v's, kappas, thetas, sigmas >= 0; rhos in [-0.999, 0.999]
lb = [1e-4, 1e-4, 1e-4, 1e-4, -0.999,
      1e-4, 1e-4, 1e-4, 1e-4, -0.999]
ub = [2.0, 10.0, 2.0, 5.0, 0.999,
      2.0, 10.0, 2.0, 5.0, 0.999]

opt = least_squares(
    residuals,
    x0,
    bounds=(lb, ub),
    verbose=2,
    xtol=1e-6,
    ftol=1e-6,
    max_nfev=300
)

# 6) Print results 
opt_x = opt.x
print("Calibrated double Heston parameters:")
print(f" Factor 1: v01={opt_x[0]:.4f}, kappa1={opt_x[1]:.4f}, theta1={opt_x[2]:.4f}, sigma1={opt_x[3]:.4f}, rho1={opt_x[4]:.4f}")
print(f" Factor 2: v02={opt_x[5]:.4f}, kappa2={opt_x[6]:.4f}, theta2={opt_x[7]:.4f}, sigma2={opt_x[8]:.4f}, rho2={opt_x[9]:.4f}")


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.0716e+06                                    5.20e+06    
       1              2         7.9663e+04      9.92e+05       6.75e-01       4.84e+05    
       2              3         2.1465e+04      5.82e+04       3.74e-01       1.15e+05    
       3              4         9.5263e+03      1.19e+04       1.80e-01       5.22e+04    
       4              5         9.0250e+03      5.01e+02       9.16e-01       3.77e+06    
       5              6         6.0348e+03      2.99e+03       9.13e-02       6.84e+05    
       6              7         5.8137e+03      2.21e+02       2.23e-01       1.41e+06    
       7              8         5.0594e+03      7.54e+02       9.08e-02       4.15e+05    
       8              9         4.6953e+03      3.64e+02       1.81e-01       8.62e+05    
       9             10         4.2520e+03      4.43e+02       7.57e-02       3.55e+05    

In [14]:
print("Prix Black-Sholes, M=1Y, S=100%:", bs_call_price(S0,S0,r,q,0.1639,1.0))

params_calibrated = {
    "S0": 100,
    "K" : 100,
    "r":  r,
    "q": q,
    "v01": opt_x[0],
    "kappa1": opt_x[1],
    "theta1": opt_x[2],
    "sigma1": opt_x[3],
    "rho1": opt_x[4],
    "v02": opt_x[5],
    "kappa2": opt_x[6],
    "theta2": opt_x[7],
    "sigma2": opt_x[8],
    "rho2": opt_x[9],
    "T": 1.0
}
call, put = price_call_put(params_calibrated)
print(f"Call Price (calibrated): {call:.4f}")

Prix Black-Sholes, M=1Y, S=100%: 242.4783779478173
Call Price (calibrated): 6.3809


# Binary Option Pricing

In [15]:
def double_heston_price_binary(S0,K,T,params, N=10000, U_max=1000):
    """
    Calculate binary option prices using the Heston model
    
    Parameters:
    params (dict): Model parameters
    N (int): Number of integration points
    U_max (float): Upper integration limit
    
    Returns:
    float: Binary option price
    """
    params["S0"] = S0
    params["K"]  = K
    params["T"]  = T
    r  = params["r"]

    if N % 2 == 1:
        N += 1

    u = np.linspace(1e-10, U_max, N + 1)
    du = u[1] - u[0]
    lnK = np.log(K)

    # Calculate characteristic function
    phi_u = double_heston_charfunc(u, params)

    # For binary options, we only need P2
    integrand = np.real(np.exp(-1j * u * lnK) * phi_u / (1j * u))

    # Simpson's weights
    weights = np.ones(N + 1)
    weights[1:-1:2] = 4.0
    weights[2:-2:2] = 2.0

    # Probability calculation
    P2 = 0.5 + (du / (3.0 * np.pi)) * np.sum(weights * integrand)

    # Binary option payoffs
    binary_call = np.exp(-r * T) * P2
    binary_put = np.exp(-r * T) * (1 - P2)

    return binary_call, binary_put



In [16]:
binary_call,binary_put = double_heston_price_binary(100,100,1,params)
print(f"Binary Call Price: {binary_call:.4f}")
print(f"Binary Put Price: {binary_put:.4f}")

Binary Call Price: 0.5363
Binary Put Price: 0.4498
